In [ ]:
# set up Dependencies
import pandas as pd
import numpy as np
import requests
import time
import os
import matplotlib.pyplot as plt
from scipy.stats import linregress
from citipy import citipy

# import api key from environment variables
weather_api_key = os.environ.get('WEATHER_API_KEY')

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set(set_size) of random lat and lng combinations
set_size=250
lats = np.random.uniform(lat_range[0], lat_range[1], size=set_size)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=set_size)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1])
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(len(cities))

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it's being processed (with the city number and city name).

In [ ]:
# Build url for weather api call. Set units to imperial
base_url = "https://api.openweathermap.org/data/2.5/weather?"
units = "&units=imperial"
API_KEY = "&appid="+str(weather_api_key)

# create lists for data from api responses
city_names = []
city_lats = []
city_lons = []
max_temps = []
humidities = []
cloudies = []
wind_spds = []
countries = []
dates = []

# --------- Record Processing Loop ----------
for city in cities:
    # wait 1 second between each API call so calls/per/minute-limit is not reached
    time.sleep(1)

    # print process log
    print("Processing Record " + str((cities.index(city) + 1)) + " of " + str(len(cities)) + " | " + city.city_name)
    
    # build URL for each city
    URL = base_url + "q=" + city.city_name + "," + city.country_code + units + API_KEY
    
    # HTTP request
    response = requests.get(URL)

    # checking the status code of the request
    if response.status_code == 200:
        city_names.append(city.city_name)
        countries.append(city.country_code)

        # getting data in the json format
        data = response.json()
        # getting the coord, main, clouds, wind dict blocks
        main = data['main']
        coord = data['coord']
        clouds = data['clouds']
        wind = data['wind']
        # getting city coordinates
        city_lats.append(coord['lat'])
        city_lons.append(coord['lon'])
        # getting max temperature
        max_temps.append(main['temp_max'])
        # getting the humidity
        humidities.append(main['humidity'])     
        # getting cloudiness %
        cloudies.append(clouds['all'])
        # getting wind speed
        wind_spds.append(wind['speed'])
        # getting time of data calculation, unix, UTC
        dates.append(data['dt'])
    else:
        # showing the error message
        print("Error in the HTTP request")

# --------- end ---------

In [ ]:
# create a dictionary from lists that were filled from api calls
weather_dict = {"City" : city_names, "Lat" : city_lats, "Lng" : city_lons, 
"Max Temps" : max_temps, "Humidity" : humidities, "Cloudiness" : cloudies,
"Wind Speed" : wind_spds, "Country" : countries, "Date" : dates}

# convert dictionary to dataframe
weather_df = pd.DataFrame(weather_dict)

print(weather_df.head())

In [ ]:
# if there are any cities with humidity > 100, drop them. Otherwise data is clean.
if(weather_df.index[weather_df["Humidity"] > 100].empty):
    clean_weather_df = weather_df  
else:
    clean_weather_df = weather_df.drop(
        weather_df.index[weather_df["Humidity"] > 100])

# export to csv
clean_weather_df.to_csv('part1.csv', index=False)